In [1548]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import os

In [1549]:
"""passar o caminho do arquiv no windows C:\estimadoresteste\pedidos.csv"""
drive_path = "informar o caminho do arquivo"

In [1550]:
data = pd.read_csv(r'C:\estimadoresteste\pedidos.csv',sep=";")
data['mesvenda'] = pd.to_datetime(data['data_cadastro'])
data.columns

Index(['cod_pedido', 'ref_contrato', 'data_cadastro', 'ref_produto',
       'valor_vendido', 'disconto', 'quantidade', 'ref_categoria',
       'valordavenda', 'categoria', 'mesvenda'],
      dtype='object')

In [1551]:

data['data_cadastro'] = pd.to_datetime(data['data_cadastro']).dt.month
data['valor_vendido'] = data['valor_vendido'].apply(lambda k: str(k).replace('.','').replace(',','.'))
data['valordavenda'] = data['valordavenda'].apply(lambda k: str(k).replace('.','').replace(',','.'))
data['valordavenda'] = data['valordavenda'].astype(float)
data['valor_vendido'] = data['valor_vendido'].astype(float)
data['disconto'] = data['disconto'].apply(lambda k: str(k).replace('.','').replace(',','.'))
data['disconto'] = data['disconto'].astype(float)
data['quantidade'] = data['quantidade'].astype(int)
data['ref_produto'] = data['ref_produto'].astype(int)

In [1552]:
data['ref_produto']

0    706
1    709
2    700
Name: ref_produto, dtype: int32

In [1553]:
def adiciona_desconto_data(df):
 
  descontos = []
  for date in df.mesvenda:
    # 'descontos' aplicados
    if date.month in [5, 6, 7, 11, 12]:
      descontos.append(np.random.random() * 0.5)
    else:
      descontos.append(0.0)
  return descontos


# coluna da data do cadastro da venda
data.data_cadastro = pd.to_datetime(data.data_cadastro)

#Adicionar dados de desconto simulados

data['data_cadastro'] = adiciona_desconto_data(data)
# Calculate price of each item from Sales and Quantity
data['valor_vendido'] = data.valordavenda / data.quantidade


#data = data.drop( 'Unnamed: 0', eixo = 1 )

In [1554]:
data.columns

Index(['cod_pedido', 'ref_contrato', 'data_cadastro', 'ref_produto',
       'valor_vendido', 'disconto', 'quantidade', 'ref_categoria',
       'valordavenda', 'categoria', 'mesvenda'],
      dtype='object')

In [1555]:
data.ref_categoria.unique()

array([12], dtype=int64)

In [1556]:
"""


Aqui agrupo e agrego as transações de forma que para cada data e

# - quantidade itens\ total vendida
# - Valor Total de Vendas
# - Desconto médio em percentual
# - Preço médio do item

"""


'\n\n\nAqui agrupo e agrego as transações de forma que para cada data e\n\n# - quantidade itens\\ total vendida\n# - Valor Total de Vendas\n# - Desconto médio em percentual\n# - Preço médio do item\n\n'

In [1557]:
product_grain = 'categoria'


"""


Aqui agrupo e agrego as transações de forma que para cada data e

# - quantidade itens\ total vendida
# - Valor Total de Vendas
# - Desconto médio em percentual
# - Preço médio do item

"""


vendas_data_Categoria = data.groupby(['data_cadastro',product_grain]).agg({'quantidade': 'sum','valordavenda': 'sum','disconto': np.average,'valor_vendido': np.average}).reset_index()
# Lets bring the day and month into separate columns
vendas_data_Categoria['day'] = pd.to_datetime(data['data_cadastro'],errors='coerce').dt.day
vendas_data_Categoria['month'] = pd.to_datetime(data['data_cadastro'],errors='coerce').dt.month


In [1558]:
vendas_data_Categoria

,data_cadastro,categoria,quantidade,valordavenda,disconto,valor_vendido,day,month
0,0.0,sala,68,68940.47,7.666667,1533.156667,1,1


In [1559]:
encoder = LabelEncoder().fit(vendas_data_Categoria[product_grain].values) 
vendas_data_Categoria[product_grain] = encoder.transform(vendas_data_Categoria[product_grain]) 
vendas_data_Categoria

,data_cadastro,categoria,quantidade,valordavenda,disconto,valor_vendido,day,month
0,0.0,0,68,68940.47,7.666667,1533.156667,1,1


In [1560]:
vendas_data_Categoria.columns

Index(['data_cadastro', 'categoria', 'quantidade', 'valordavenda', 'disconto',
       'valor_vendido', 'day', 'month'],
      dtype='object')

In [1561]:
rf = RandomForestRegressor() 

X = vendas_data_Categoria[[product_grain, 'disconto' , 'valor_vendido' , 'day' , 'month']] 
y = vendas_data_Categoria[ 'quantidade' ] 

rf = rf.fit(X, y)

In [1562]:
# Prever a quantidade nos dados
vendas_data_Categoria[ 'predicted_quantity' ] = rf.predict(X) 
# Calcular o erro absoluto médio nas previsões
vendas_data_Categoria[ 'error' ] = abs (vendas_data_Categoria[ 'quantidade' ] - vendas_data_Categoria.predicted_quantity) 


In [1563]:
def optimize_quantity_out(model, df, product_grain, product_grain_value, day, month):


  item_price_mean = df[df[product_grain] == product_grain_value].valor_vendido.mean()

  # obtem precos
  potential_price = np.linspace(item_price_mean * 0.25, item_price_mean * 2, 50)
  # descontos
  potential_discount = np.linspace(0, 0.7, 20)

  # combinando preços e descontos
  samples = []
  for d in potential_discount:
    for p in potential_price:
      print(potential_price)
      sample = {
          product_grain: product_grain_value,
          'disconto': d,
          'valor_vendido': p,
          'day': day,
          'month': month
         
        
      }
      print(product_grain)
      samples.append(sample)

  # modelo treinado em amostras virtuais
  samples = pd.DataFrame(samples)
  samples['Previsao'] = model.predict(samples)
  # parâmetros ideais de tags
  samples['is_optimal'] = [True if s == max(samples.Previsao) else False for s in samples.Previsao]
  return samples




In [1564]:
categories = vendas_data_Categoria.categoria.unique()
categories

array([0])

In [1565]:
vendas_data_Categoria

,data_cadastro,categoria,quantidade,valordavenda,disconto,valor_vendido,day,month,predicted_quantity,error
0,0.0,0,68,68940.47,7.666667,1533.156667,1,1,68.0,0.0


In [1566]:
vendas_data_Categoria

,data_cadastro,categoria,quantidade,valordavenda,disconto,valor_vendido,day,month,predicted_quantity,error
0,0.0,0,68,68940.47,7.666667,1533.156667,1,1,68.0,0.0


In [1567]:
import random

"""

iformar o periodo que deseja fazer

"""
day = 3
month = 5

#
sku = random.choice(categories)

optimal = optimize_quantity_out(rf, vendas_data_Categoria, product_grain, sku, day, month)



[ 383.28916667  438.0447619   492.80035714  547.55595238  602.31154762
  657.06714286  711.8227381   766.57833333  821.33392857  876.08952381
  930.84511905  985.60071429 1040.35630952 1095.11190476 1149.8675
 1204.62309524 1259.37869048 1314.13428571 1368.88988095 1423.64547619
 1478.40107143 1533.15666667 1587.9122619  1642.66785714 1697.42345238
 1752.17904762 1806.93464286 1861.6902381  1916.44583333 1971.20142857
 2025.95702381 2080.71261905 2135.46821429 2190.22380952 2244.97940476
 2299.735      2354.49059524 2409.24619048 2464.00178571 2518.75738095
 2573.51297619 2628.26857143 2683.02416667 2737.7797619  2792.53535714
 2847.29095238 2902.04654762 2956.80214286 3011.5577381  3066.31333333]
categoria
[ 383.28916667  438.0447619   492.80035714  547.55595238  602.31154762
  657.06714286  711.8227381   766.57833333  821.33392857  876.08952381
  930.84511905  985.60071429 1040.35630952 1095.11190476 1149.8675
 1204.62309524 1259.37869048 1314.13428571 1368.88988095 1423.64547619
 14

In [1568]:
optimal['valor_vendido'] = optimal['valor_vendido'].apply(lambda k: str(k).replace(",",""))
optimal['valor_vendido'] = optimal['valor_vendido'].apply(lambda k: round(float(k),2))


In [1569]:
import plotly.express as px
import plotly.graph_objects as go

# gerando grafico de dispersao
fig = px.scatter_3d(optimal, x='valor_vendido', y='disconto', z='Previsao', color='is_optimal', width=800, height=800)

fig.show()